In [1]:
import pandas as pd
from bokeh.models import (ColumnDataSource, HoverTool, DataTable, TableColumn, DataCube, GroupingInfo, GroupFilter, CDSView, TabPanel, Tabs)
from bokeh.plotting import figure, show
from bokeh.layouts import column
import pandas as pd
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [14]:
final_status = pd.read_pickle("/media/user/files/code/imdb/data/status.pickle")
final_status

,watched,priority,netflix,prime,averageRating,numVotes,primaryTitle,originalTitle,startYear,runtimeMinutes,...,genre_Crime,genre_Drama,genre_Family,genre_Fantasy,genre_History,genre_Horror,genre_Mystery,genre_Romance,genre_Sport,genre_Thriller
tconst,,,,,,,,,,,,,,,,,,,,,
tt0025316,False,True,<NA>,<NA>,8.1,116741,It Happened One Night,It Happened One Night,1934,105,...,False,False,False,False,False,False,False,True,False,False
tt0092005,False,True,False,False,8.1,464056,Stand by Me,Stand by Me,1986,89,...,False,True,False,False,False,False,False,False,False,False
tt0355702,False,True,False,False,7.1,59561,Lords of Dogtown,Lords of Dogtown,2005,107,...,False,True,False,False,False,False,False,False,True,False
tt0367594,False,True,False,False,6.7,550860,Charlie and the Chocolate Factory,Charlie and the Chocolate Factory,2005,115,...,False,False,True,False,False,False,False,False,False,False
tt0205000,False,True,False,False,5.7,82525,Deuce Bigalow: Male Gigolo,Deuce Bigalow: Male Gigolo,1999,88,...,False,False,False,False,False,False,False,True,False,False
tt0240515,False,True,False,False,4.7,53048,Freddy Got Fingered,Freddy Got Fingered,2001,87,...,False,False,False,False,False,False,False,False,False,False
tt0048021,False,False,True,False,8.1,38035,Rififi,Du rififi chez les hommes,1955,118,...,True,True,False,False,False,False,False,False,False,True
tt2005151,True,True,True,True,7.1,277652,War Dogs,War Dogs,2016,114,...,True,False,False,False,False,False,False,False,False,False
tt0081505,True,False,True,True,8.4,1165622,The Shining,The Shining,1980,146,...,False,True,False,False,False,True,False,False,False,False


In [15]:
final_status["priority"] = final_status["priority"].map({True: "y", False: "n"})

In [16]:
source = ColumnDataSource(final_status[~final_status["watched"]])
# filtered_source = ColumnDataSource(final_status[~final_status["watched"] & final_status["priority"]])


filter_priority = GroupFilter(column_name='priority', group="y")
view_priority = CDSView(filter=filter_priority)
view_all = CDSView()

TOOLTIPS = [
    ("title", "@primaryTitle"),
    ("year", "@startYear")
]

In [17]:
# Create a Bokeh figure
fig_all = figure(title="Unwatched movies",
        x_axis_label="Number of Votes",
        y_axis_label="Average Rating",
        width=500, height=500, y_range=(0,10),
        tooltips=TOOLTIPS)

# Add scatter plot
fig_all.circle(x="numVotes", y="averageRating", radius=5000, source=source, name="all", view=view_all)

show(fig_all)

In [18]:
# Create a Bokeh figure
fig_prio = figure(title="Unwatched priority movies",
        x_axis_label="Number of Votes",
        y_axis_label="Average Rating",
        width=500, height=500, y_range=(0,10),
        tooltips=TOOLTIPS)

# Add scatter plot
fig_prio.circle(x="numVotes", y="averageRating", radius=5000, source=source, name="prio", view=view_priority)

show(fig_prio)

In [ ]:
hover_all = fig_all.select_one(HoverTool)
hover_all.point_policy = "follow_mouse"
# And we reference the mayor in the tooltip
hover_all.tooltips = [
    ("title", "@primaryTitle"),
    ("year", "@startYear")
]

hover_prio = fig_prio.select_one(HoverTool)
hover_prio.point_policy = "follow_mouse"
# And we reference the mayor in the tooltip
hover_prio.tooltips = [
    ("title", "@primaryTitle"),
    ("year", "@startYear")
]


all_tabs = Tabs(tabs=[
    TabPanel(child=fig_prio, title="priority movies"),
    TabPanel(child=fig_all, title="all movies"),
])

show(all_tabs)

In [21]:
##################################################################################
genre_columns = [ x for x in list(final_status.columns) if "genre" in x ]

genres = final_status[genre_columns].replace(False, pd.NA).stack().reset_index().drop(0, axis=1).rename(columns={"level_1": "genre"})
genres["genre"] = genres["genre"].str.replace("genre_","")
genres = genres.set_index("tconst")

display_status = final_status[~final_status["watched"]].drop(columns=genre_columns)
display_status = display_status.join(genres).groupby("genre").head(10).sort_values(["genre", "averageRating"], ascending=[True, False])
display_status = display_status[["averageRating", "primaryTitle", "startYear", "genre"]].round({"averageRating": 1})

top10 = ColumnDataSource(display_status)
columns = [
        TableColumn(field="tconst", title="tconst"),
        TableColumn(field="genre", title="genre"),
        TableColumn(field="primaryTitle", title="title"),
        TableColumn(field="startYear", title="year"),
        TableColumn(field="averageRating", title="rating"),
    ]

grouping = [
    GroupingInfo(getter="genre")
]

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

data_cube = DataCube(source=top10, columns=columns, grouping=grouping, target=target, width=500, height=500)

show(column(all_tabs, data_cube))

